# ETL
En este ETL lo que realizamos en cargar los dataset en formato Json , los leemos mediante una función creada y como dos dataset tienen una columna anidadd , tambien creamos una funcion para desanidar la misma.
los dataset que haran uso de esta funcion de desanidar son 'australian_users_items.json' y 'australian_user_reviews.json'


In [1]:


#Funcion para leer archivos Json

import ast
import pandas as pd

def leer_json(ruta_archivo):
    """
    Carga un archivo JSON y devuelve un DataFrame.

    Parameters:
        ruta_archivo (str): La ruta al archivo JSON.

    Returns:
        pd.DataFrame: El DataFrame cargado desde el archivo JSON.
    """
    filas = []

    # Abrimos el archivo en modo lectura con la codificación "MacRoman"
    with open(ruta_archivo, encoding="MacRoman") as archivo:
        # Iteraramos sobre cada línea en el archivo
        for linea in archivo.readlines():
            # Evaluaamos la línea como una expresión de Python y agregamos a la lista
            filas.append(ast.literal_eval(linea))

    # Creamos y devolvemos un DataFrame a partir de la lista de filas
    return pd.DataFrame(filas)


def desanidar_columna(df_entrada, columna_anidada):
    """
    Expande una columna anidada en un DataFrame.

    Parameters:
    - df_entrada (pd.DataFrame): DataFrame de entrada con la columna anidada.
    - columna_anidada (str): Nombre de la columna anidada que se va a expandir.

    Returns:
    pd.DataFrame: Nuevo DataFrame con la columna anidada expandida.
    """
    # Inicializa  una lista vacía para almacenar los registros expandidos.
    registros_expandidos = []

    # Itera sobre cada fila del DataFrame de entrada.
    for indice, fila in df_entrada.iterrows():
        # Extraemos los datos base de la fila eliminando la columna anidada y convirtiéndolos a un diccionario.
        datos_base = fila.drop(columna_anidada).to_dict()
        # Obtenemos la lista de datos anidados de la columna especificada.
        datos_anidados_lista = fila[columna_anidada]

        # Iteramos sobre cada conjunto de datos anidados.
        for datos_anidados in datos_anidados_lista:
            # Crea un nuevo registro combinando los datos base y los datos anidados.
            nuevo_registro = {**datos_base, **datos_anidados}
            # Agrega el nuevo registro a la lista de registros_expandidos.
            registros_expandidos.append(nuevo_registro)

    # Creamos un nuevo DataFrame a partir de la lista de registros_expandidos y lo devolvemos.
    df_expandido = pd.DataFrame(registros_expandidos)
    return df_expandido



In [2]:
#leemos el archivo items

# Ruta del archivo JSON
ruta_json ='australian_users_items.json'


df_user_items = leer_json(ruta_json)# Cargo el archivo JSON en un DataFrame


df_user_items = desanidar_columna(df_user_items, "items") # se desanida las columnas anidadas
df_user_items.head()

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6,0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0,0
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7,0
3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40,Deathmatch Classic,0,0
4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50,Half-Life: Opposing Force,0,0


In [3]:
df_user_items.info()# verificamos la informacion de las columnas 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 8 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   items_count       int64 
 2   steam_id          object
 3   user_url          object
 4   item_id           object
 5   item_name         object
 6   playtime_forever  int64 
 7   playtime_2weeks   int64 
dtypes: int64(3), object(5)
memory usage: 314.5+ MB


In [5]:
#eliminar columnas innecesarias
df_user_items= df_user_items.drop(['steam_id', 'playtime_2weeks'], axis=1)

In [6]:
#Guardamos el dataframe en formato csv para su posterior uso

df_user_items.to_parquet('user_items.parquet', index=False)

In [12]:
#leemos el archivo reviews

# Ruta del archivo JSON
ruta_json ='australian_user_reviews.json'

# Cargo el archivo JSON en un DataFrame
df_user_reviews = leer_json(ruta_json)


df_user_reviews = desanidar_columna(df_user_reviews, "reviews") #se desanida las columnas anidadas
df_user_reviews.head()

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [8]:
df_user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      59305 non-null  object
 1   user_url     59305 non-null  object
 2   funny        59305 non-null  object
 3   posted       59305 non-null  object
 4   last_edited  59305 non-null  object
 5   item_id      59305 non-null  object
 6   helpful      59305 non-null  object
 7   recommend    59305 non-null  bool  
 8   review       59305 non-null  object
dtypes: bool(1), object(8)
memory usage: 3.7+ MB


In [14]:
df_user_reviews.to_parquet('user_items_reviews.parquet', index=False) #guardamos en tipo parquet 

In [10]:
#Leemos el archivo output_steam_games
import pandas as pd
import json


steams_games=[]
with open('output_steam_games.json', 'r') as file:
    for line in file:
        k = json.loads(line)
        steams_games.append(k)


df_steam_games = pd.DataFrame(steams_games)

df_steam_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 11.9+ MB


In [11]:
df_steam_games.to_csv('steam_games.csv', index=False)#guardamos en CSV